# Simple RNN Model

## Introduction
In this notebokk, we will use a recurrent neural network (RNN) to determine whether a text sequence of indefinite length contains positive or negative emotion. An RNN takes a sequence of words, X = {x_1, ..., x_T} as input. At each time step, the model generates a hidden state, h, from the current input word and the previous hidden state.

                        h_t = RNN(x_t, h_t-1)
                        
The final hidden state, h_T, will be generated from the last word in the sequence with encoded historical information from all the previous steps. Once obtaining the final state, we feed it to a dense layer (also knowns as a fully connected layer) to predict a sentiment output, Y_hat = f(h_T).

In [4]:
# Set the randomw seed for reproducibility
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [07:19<00:00, 192kB/s] 


In [7]:
print(raw_train_data)

In [13]:
raw_train_data = list(raw_train_data)
raw_test_data = list(raw_test_data)

In [14]:
print(f'Number of training examples: {len(raw_train_data):,}')
print(f'Number of testing examples: {len(raw_test_data):,}')

Number of training examples: 25,000
Number of testing examples: 25,000


In [12]:
print(raw_train_data[0])

(0, 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between

In [17]:
def get_train_valid_split(raw_train_data, split_ratio = 0.7):
    raw_train_data = list(raw_train_data)
    random.shuffle(raw_train_data)
    
    n_train_examples = int(len(raw_train_data) * split_ratio)
    
    train_data = raw_train_data[:n_train_examples]
    valid_data = raw_train_data[n_train_examples:]
    
    return train_data, valid_data 

In [18]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [19]:
raw_train_data = list(raw_train_data)
raw_valid_data = list(raw_valid_data)

In [20]:
print(f'Number of training examples: {len(raw_train_data):,}')
print(f'Number of validation examples: {len(raw_valid_data):,}')
print(f'Number of testing examples: {len(raw_test_data):,}')

Number of training examples: 17,500
Number of validation examples: 7,500
Number of testing examples: 25,000


In [21]:
class Tokenizer:
    def __init__(self, tokenize_fn='basic_english', lower=True, max_length=None):
        self.tokenize_func = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, s):
        tokens = self.tokenize_func(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.max_length is not None:
            tokens = tokens[:self.max_length]
            
        return tokens

In [22]:
max_length = 500

tokenizer = Tokenizer(max_length = max_length)

In [23]:
s = "hello world!"
print(tokenizer.tokenize(s))

['hello', 'world', '!']


In [24]:
print(s.split())

['hello', 'world!']


In [26]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
    token_freq = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freq.update(tokens)
    
    vocab = torchtext.vocab.Vocab(token_freq, **vocab_kwargs)
    
    return vocab

In [27]:
max_size = 25000

vocab = build_vocab_from_data(raw_train_data, tokenizer, max_size = max_size)

In [28]:
vocab.freqs.most_common(20)

[('the', 211890),
 ('.', 208427),
 (',', 173201),
 ('a', 103447),
 ('and', 103052),
 ('of', 91695),
 ('to', 84931),
 ("'", 83805),
 ('is', 67948),
 ('it', 61322),
 ('in', 58757),
 ('i', 57420),
 ('this', 49214),
 ('that', 46061),
 ('s', 38864),
 ('was', 31341),
 ('as', 29171),
 ('movie', 28776),
 ('for', 27903),
 ('with', 27680)]

In [30]:
vocab.itos[:10]

['<unk>', '<pad>', 'the', '.', ',', 'a', 'and', 'of', 'to', "'"]

In [31]:
def process_raw_data(raw_data, tokenizer, vocab):
    #raw_data = [(label, text) for (label, text) in raw_data]
    
    text_transform = sequential_transforms(tokenizer.tokenize, 
                                           vocab_func(vocab),
                                           totensor(dtype=torch.long))
    
    label_transform = sequential_transforms(totensor(dtype=torch.long))
    
    transforms = (label_transform, text_transform)
    
    dataset = TextClassificationDataset(raw_data,
                                        vocab,
                                        transforms)
    
    return dataset

In [32]:
train_data = process_raw_data(raw_train_data, tokenizer, vocab)

In [33]:
len(train_data)

17500

In [34]:
train_data

In [36]:
label, indexes = train_data[0]

print(indexes)

tensor([  610,  8612,    10,     5,    60,   221,     6,     5,    60,     0,
          172,     3,    32,    91,    19,     9,   330,     7,  1487,     9,
           17,     2,    35,    13,   414,   128,     4,     6,    11,   293,
            5,   210,     7,   116,    21,   111,   632,  1851,     7,   125,
         1427,    18,    39,    84,    90,  2552,  1114,     4,     6,    44,
          131,     2,  1851,     7,     2,   490,     3,    44,    10,     9,
         6126,     9,    72,    43,     2,   428,   496,     8,   293,     2,
          348,     7,     2,   490,     8,     2,   584,   831,   481,     3,
            2,  4673,   111,    30,   119,  1783,     4,    35,  3138,     6,
           35,  2349,    42,   854,    21,     5, 17557,  5565,  1586,     3,
            2,   560,     7,    41,   174,  3138,  2038,  1996,    42,  1431,
            8,    34,    41,  2004,  2289,     7,     2,  6375,  5693,   346,
            0,     2,  3138,  2140,    12,     2,   348,     6, 

In [38]:
print([vocab.itos[i] for i in indexes])

['david', 'mamet', 'is', 'a', 'very', 'interesting', 'and', 'a', 'very', '<unk>', 'director', '.', 'his', 'first', 'movie', "'", 'house', 'of', 'games', "'", 'was', 'the', 'one', 'i', 'liked', 'best', ',', 'and', 'it', 'set', 'a', 'series', 'of', 'films', 'with', 'characters', 'whose', 'perspective', 'of', 'life', 'changes', 'as', 'they', 'get', 'into', 'complicated', 'situations', ',', 'and', 'so', 'does', 'the', 'perspective', 'of', 'the', 'viewer', '.', 'so', 'is', "'", 'homicide', "'", 'which', 'from', 'the', 'title', 'tries', 'to', 'set', 'the', 'mind', 'of', 'the', 'viewer', 'to', 'the', 'usual', 'crime', 'drama', '.', 'the', 'principal', 'characters', 'are', 'two', 'cops', ',', 'one', 'jewish', 'and', 'one', 'irish', 'who', 'deal', 'with', 'a', 'racially', 'charged', 'area', '.', 'the', 'murder', 'of', 'an', 'old', 'jewish', 'shop', 'owner', 'who', 'proves', 'to', 'be', 'an', 'ancient', 'veteran', 'of', 'the', 'israeli', 'independence', 'war', '<unk>', 'the', 'jewish', 'identity

In [39]:
valid_data = process_raw_data(raw_valid_data, tokenizer, vocab)
test_data = process_raw_data(raw_test_data, tokenizer, vocab)

In [40]:
print(vocab['<unk'])

0


In [105]:
class Collator:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    def collate(self, batch):
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)
        
        text = nn.utils.rnn.pad_sequence(text, padding_value = self.pad_idx)
        
        return labels, text

In [107]:
pad_token = '<pad>'
pad_idx = vocab[pad_token]

collator = Collator(pad_idx)

In [108]:
batch_size = 256

train_iterator = torch.utils.data.DataLoader(train_data,
                                             batch_size,
                                             shuffle = True,
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data,
                                             batch_size,
                                             shuffle = False,
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data,
                                             batch_size,
                                             shuffle = False,
                                             collate_fn = collator.collate)

In [97]:
train_iterator

In [98]:
len(train_iterator)

69

In [100]:
69 * 256

17664

In [47]:
class NBOW(nn.Module):
    def __init__(self, input_dim, emb_dim, output_dim, pad_idx):
        super(NBOW, self).__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.fc = nn.Linear(emb_dim, output_dim)
        
    def forward(self, text):
        # text = [seq_len, batch_size]
        embedded = self.embedding(text)
        
        # embedded = [seq_len, batch_size, emb_dim]
        
        pooled = embedded.mean(0)
        
        # pooled = [batch_size, emb_dim]
        
        prediction = self.fc(pooled)
        
        # prediction = [batch_size, output_dim]
        
        return prediction

In [48]:
input_dim = len(vocab)

In [49]:
input_dim

25002

In [50]:
emb_dim = 100
output_dim = 2

model = NBOW(input_dim, emb_dim, output_dim, pad_idx)

In [51]:
model.parameters

<bound method Module.parameters of NBOW(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (fc): Linear(in_features=100, out_features=2, bias=True)
)>

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [55]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,402 trainable parameters


In [57]:
[p.numel() for p in model.parameters() if p.requires_grad]

[2500200, 200, 2]

In [66]:
glove = torchtext.experimental.vectors.GloVe(name = '6B', dim = '100')

glove.6B.zip: 100%|██████████| 862M/862M [1:06:41<00:00, 215kB/s]   


In [67]:
glove['ai']

tensor([ 0.0451,  0.0083,  1.0102,  0.0388, -0.3000,  0.1082,  0.2669, -0.1797,
         0.6793, -0.0844,  1.0620,  0.7680, -0.1149,  0.2395,  0.0772,  0.2139,
        -0.0788,  0.4310, -0.1115,  0.6688,  0.3650,  0.6718,  0.0884,  0.6221,
        -0.1799,  0.8679,  0.3089, -0.5903,  0.6090, -0.0580, -0.8839,  0.7195,
         0.8916,  0.4024,  0.2230,  0.0967, -0.4288, -0.0639,  0.4010, -1.1798,
         0.4766,  0.4740, -0.2950, -0.3705,  0.1640, -1.0603, -0.3326,  0.0624,
         0.1529, -0.3260, -0.3132,  0.0538, -0.1375,  0.0059,  0.1142, -0.6263,
         0.6887,  0.9929,  0.2639,  0.9296, -0.1899,  0.2262, -1.0052,  0.6435,
         0.3932,  0.7644, -0.0143,  0.3956, -0.1160,  0.3971,  0.3830,  0.2465,
         0.0164,  0.1025, -0.1576,  0.6056, -0.1152,  0.2444, -0.1456,  0.0602,
         0.4137, -0.5334, -0.8109, -0.1220, -1.6253,  0.8068, -0.5653,  0.0348,
        -0.1021,  0.1404,  0.7914, -0.3282,  0.6349,  0.7546,  0.2400, -0.2933,
        -1.0098, -1.0907, -0.1969, -0.03

In [68]:
len(glove['ai'])

100

In [69]:
glove_vocab = glove.vectors.get_stoi()

In [70]:
'the' in glove_vocab

True

In [86]:
def get_pretrained_embedding(initial_embedding, pretrained_vectors, vocab, unk_token):
    
    pretrained_embedding = torch.FloatTensor(initial_embedding.weight.clone()).detach()    
    pretrained_vocab = pretrained_vectors.vectors.get_stoi()
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        if token in pretrained_vocab:
            pretrained_vector = pretrained_vectors[token]
            pretrained_embedding[idx] = pretrained_vector
        else:
            unk_tokens.append(token)
        
    return pretrained_embedding, unk_tokens

In [77]:
unk_token = '<unk>'
# no detach
pretrained_embedding, unk_tokens = get_pretrained_embedding(model.embedding, glove, vocab, unk_token)

25002
tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [ 0.0914,  1.5196,  0.4670,  ...,  0.6393, -0.0332,  0.0185],
        [-0.6290,  0.4650, -0.7165,  ..., -1.3171,  2.0381, -2.0497],
        [-1.1222, -0.0240, -1.0878,  ..., -0.4948, -0.3874,  0.0339]],
       grad_fn=<SliceBackward>)


In [79]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(model.embedding, glove, vocab, unk_token)

25002
tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [ 0.0914,  1.5196,  0.4670,  ...,  0.6393, -0.0332,  0.0185],
        [-0.6290,  0.4650, -0.7165,  ..., -1.3171,  2.0381, -2.0497],
        [-1.1222, -0.0240, -1.0878,  ..., -0.4948, -0.3874,  0.0339]])


In [80]:
model.embedding.weight.data

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [ 0.0914,  1.5196,  0.4670,  ...,  0.6393, -0.0332,  0.0185],
        [-0.6290,  0.4650, -0.7165,  ..., -1.3171,  2.0381, -2.0497],
        [-1.1222, -0.0240, -1.0878,  ..., -0.4948, -0.3874,  0.0339]])

In [88]:
len(unk_tokens)

678

In [89]:
print(unk_tokens[:10])

['<unk>', '<pad>', '\x96', 'hadn', '****', '100%', 'camera-work', '*1/2', '$1', '*****']


In [91]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [90]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [92]:
model.embedding.weight.shape

torch.Size([25002, 100])

In [93]:
optimizer = optim.Adam(model.parameters())

In [94]:
criterion = nn.CrossEntropyLoss()

In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using: {device}')

Using: cpu


In [96]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [101]:
def train(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text in iterator:
        
        labels = labels.to(device)
        text = text.to(device)
        
        optimizer.zero_grad()
        
        predictions = model(text)
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [102]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [103]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [109]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'nbow-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.687 | Train Acc: 55.50%
	 Val. Loss: 0.678 |  Val. Acc: 63.46%
Epoch: 02 | Epoch Time: 0m 21s
	Train Loss: 0.666 | Train Acc: 65.38%
	 Val. Loss: 0.652 |  Val. Acc: 69.54%
Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 0.631 | Train Acc: 72.05%
	 Val. Loss: 0.613 |  Val. Acc: 73.82%
Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 0.585 | Train Acc: 76.32%
	 Val. Loss: 0.568 |  Val. Acc: 76.86%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 0.533 | Train Acc: 80.33%
	 Val. Loss: 0.522 |  Val. Acc: 80.11%
Epoch: 06 | Epoch Time: 0m 19s
	Train Loss: 0.484 | Train Acc: 83.33%
	 Val. Loss: 0.480 |  Val. Acc: 82.45%
Epoch: 07 | Epoch Time: 0m 19s
	Train Loss: 0.438 | Train Acc: 85.67%
	 Val. Loss: 0.442 |  Val. Acc: 84.57%
Epoch: 08 | Epoch Time: 0m 19s
	Train Loss: 0.399 | Train Acc: 87.28%
	 Val. Loss: 0.413 |  Val. Acc: 85.63%
Epoch: 09 | Epoch Time: 0m 20s
	Train Loss: 0.366 | Train Acc: 88.44%
	 Val. Loss: 0.388 |  Val. Acc: 86.46%
Epoch: 10 | Epoch T

In [110]:
model.load_state_dict(torch.load('nbow-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.390 | Test Acc: 85.64%


In [111]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [112]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

1.091639387595933e-05

In [113]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9999977350234985

In [114]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.7893003225326538

In [115]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.7893003225326538